In [5]:
import cv2

import sys
import os
from datetime import datetime, UTC
import threading
import queue


# Add the src directory to the sys.path list
sys.path.insert(0, os.path.abspath('../src'))
from services.event_service import add_event
from db_models.init_db import create_tables
from surveillance_daemon.object_detector import ObjectDetector
from surveillance_daemon.video_capture import VideoCapture
from surveillance_daemon.motion_detector import MotionDetector
from surveillance_daemon.video_recorder import VideoRecorder
from surveillance_daemon.notification import Notification

In [6]:
# Initialize the database session
create_tables()

# Initialize system components
object_detector = ObjectDetector(
    "../models/yolov7-tiny/yolov7-tiny.cfg",
    "../models/yolov7-tiny/yolov7-tiny.weights",
    "../data/coco.names",
)
motion_detector = MotionDetector()
notification = Notification(to_email="gunes314@gmail.com")

In [7]:
# Queue to store video paths to be processed
video_queue = queue.Queue()

def worker():
    """Worker thread that processes video files from the queue."""
    while True:
        print("Start object detection worker.")
        # Unpack the video information from the queue
        raw_video_path, video_length, event_time = video_queue.get()
        if raw_video_path is None:
            break

        # Perform object detection on the unannotated video
        detected_objects, video_path = object_detector.process_video(raw_video_path)
        print(video_path)

        # Add the event to the database
        add_event(video_path, detected_objects, event_time, video_length)

        # Remove the unannotated video
        os.remove(raw_video_path)

        # send a notification if an object is detected
        if detected_objects:
            notification.send(detected_objects)
        video_queue.task_done()

# Create and start the worker thread
num_worker_threads = 1
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

In [8]:
recording = False
event_time = None

with VideoCapture(device=0) as camera:
    while True:
        frame = camera.read_frame()
        if frame is None:
            break

        font = cv2.FONT_HERSHEY_SIMPLEX
        color = (0, 255, 0)

        # Detect motion and start recording video
        if motion_detector.detect_motion(frame):
            cv2.putText(frame, "M", (10, 30), font, 0.5, color, 2)
            if not recording:
                print("Start recording.")
                recording = True
                event_time = datetime.now(UTC)
                video_recorder = VideoRecorder(folder="/home/matrik/Yandex.Disk/CM3070_FP/code/videos", fps=camera.get_fps(), width=camera.get_frame_width(), height=camera.get_frame_height())
        
        # Record video for 5 seconds
        if recording:
            cv2.putText(frame, "R", (30, 30), font, 0.5, color, 2)
            video_recorder.write_frame(frame)
            video_length = (datetime.now(UTC) - event_time).total_seconds()
            if video_length > 5:
                print("Stop recording.")
                # Start object detection on a separate thread when video recording stops
                video_queue.put((video_recorder.get_video_path(), int(video_length), event_time))

                # Clear the video recorder object
                recording = False
                video_recorder.release()
                del video_recorder

        cv2.imshow("Object detection", frame)
        if cv2.waitKey(1) == 27:
            break

    cv2.destroyAllWindows()

OpenCV: FFMPEG: tag 0x30385056/'VP80' is not supported with codec id 139 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30385056/'VP80' is not supported with codec id 139 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30385056/'VP80' is not supported with codec id 139 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30385056/'VP80' is not supported with codec id 139 and format 'webm / WebM'
OpenCV: FFMPEG: tag 0x30385056/'VP80' is not supported with codec id 139 and format 'webm / WebM'


/home/matrik/Yandex.Disk/CM3070_FP/code/videos/2024-05-18T19:22:46_unannotated.webm
/home/matrik/Yandex.Disk/CM3070_FP/code/videos/2024-05-18T19:22:46.webm
{'tvmonitor', 'diningtable', 'bowl', 'cup', 'person'}
/home/matrik/Yandex.Disk/CM3070_FP/code/videos/2024-05-18T19:22:51_unannotated.webm
/home/matrik/Yandex.Disk/CM3070_FP/code/videos/2024-05-18T19:22:51.webm
{'cup', 'tvmonitor', 'bowl', 'person'}


Releasing camera


In [ ]:
object_detector.process_video("../videos/2024-05-15T23:14:42.avi")

In [ ]:
camera.release()
cv2.destroyAllWindows()